<a href="https://colab.research.google.com/github/FahtulIkhsan/STKI-buku/blob/main/TKI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from google.colab import drive

In [ ]:
drive.mount('/content/drive')
gr = pd.read_csv('/content/drive/MyDrive/Tugas-Tugas/Goodreads.csv')
print(gr)

Mounted at /content/drive
    Book_ID                                              Title           ISBN  \
0         1                                   The Hunger Games  9780439023481   
1         2                           The Chronicles of Narnia  9780066238500   
2         3                                           Twilight  9780316015844   
3         4  Alice's Adventure in Wonderland & Through the ...  9780451527745   
4         5                                          Divergent  9780062024039   
5         6                                  Lord of the Flies  9780140283334   
6         7                                           The Help  9780399155345   
7         8                                      The Alchemist  9780062315007   
8         9                                    Of Mice and Men  9780142000670   
9        10                                            Dracula  9780393970128   
10       11                           The Time Traveler's Wife  9781939126016   
11

In [ ]:
nltk.download('stopwords')

regexp = RegexpTokenizer('\w+')
ps = PorterStemmer()

# Make a list of english stopwords
stopwords = nltk.corpus.stopwords.words("english")

# Extend the list with your own custom stopwords
my_stopwords = ['https']
stopwords.extend(my_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def preProcess(dataframe) :
  dataframe = dataframe.astype(str).str.lower()
  dataframe = dataframe.apply(regexp.tokenize)
  dataframe = dataframe.apply(lambda x: ' '.join([word for word in x if word not in (stopwords)]))
  dataframe = dataframe.apply(lambda x: ' '.join([ps.stem(word) for word in x.split()]))

  return dataframe

for column in gr :
  gr[column] = preProcess(gr[column])

In [ ]:
#https://github.com/utahta/pyvbcode

from __future__ import division 
from struct import pack, unpack

def encode_number(number):
    """Variable byte code encode number.
    Usage:
      import vbcode
      vbcode.encode_number(128)
    """
    bytes_list = []
    while True:
        bytes_list.insert(0, number % 128)
        if number < 128:
            break
        number = number // 128
    bytes_list[-1] += 128
    return pack('%dB' % len(bytes_list), *bytes_list)

def encode(numbers):
    """Variable byte code encode numbers.
    Usage:
      import vbcode
      vbcode.encode([32, 64, 128])
    """
    bytes_list = []
    for number in numbers:
        bytes_list.append(encode_number(number))
    return b"".join(bytes_list)

def decode(bytestream):
    """Variable byte code decode.
    Usage:
      import vbcode
      vbcode.decode(bytestream)
        -> [32, 64, 128]
    """
    n = 0
    numbers = []
    bytestream = unpack('%dB' % len(bytestream), bytestream)
    for byte in bytestream:
        if byte < 128:
            n = 128 * n + byte
        else:
            n = 128 * n + (byte - 128)
            numbers.append(n)
            n = 0
    return numbers

In [ ]:
import sys
import psutil
import json
from collections import OrderedDict

def indexing(dictionary, dataframe):
  mem = psutil.virtual_memory()
  i = 0
  
  while(mem.available > 0 and i < len(dataframe.index)):
    row = dataframe.iloc[i]
    if type(row) is not str :
      row = str(row)
    tokens = row.split()
    for token in tokens:
      if token not in dictionary:
        dictionary[token] = []
      if i not in dictionary[token]:
        dictionary[token].append(i)
    i = i + 1
  
  for values in dictionary.values() :
    values.sort()
  dictionary = OrderedDict(sorted(dictionary.items()))
  return dictionary

def compressIndex(dictionary) :
  # with open("./output.txt", "w") as text_file:
  #   text_file.write(json.dumps(dictionary))
  
  with open("./posting_list.bin", "wb") as binary_file, open("./dictionary.txt", "w") as dict_file :
    for i, posting_list in dictionary.items() :
      binary_file.write(encode(posting_list))
      # dict_term = str(i)+"<"+str(binary_file.tell())+">"
      # dict_file.write(dict_term)
      dictionary[i] = binary_file.tell()
    dict_file.write(json.dumps(dictionary))

  return dictionary

invertedIndex = {}
for column in gr :
  invertedIndex = indexing(invertedIndex, gr[column])
print(invertedIndex)
invertedIndex = compressIndex(invertedIndex)
print(invertedIndex)

OrderedDict([('04', [39]), ('1', [0, 1, 3, 5, 8, 11, 23, 24, 26, 27, 31, 37, 42]), ('10', [6, 9, 42]), ('100', [22]), ('11', [10, 42]), ('12', [9, 11, 18]), ('13', [12]), ('14', [0, 13, 39]), ('15', [7, 14, 15, 25]), ('16', [15, 17, 20, 32, 42]), ('17', [13, 16]), ('18', [17]), ('1811', [22]), ('1821', [22]), ('19', [18]), ('1910', [26]), ('1911', [26]), ('1912', [12]), ('1930', [8]), ('1931', [11]), ('1932', [11]), ('1937', [8]), ('1949', [1]), ('1962', [6]), ('1986', [9]), ('1998', [11, 26]), ('1999', [5]), ('2', [1, 19, 21, 33, 42]), ('20', [19]), ('2000', [3, 27]), ('2001', [31]), ('2002', [8]), ('2003', [25]), ('2004', [47]), ('2006', [2]), ('2007', [24]), ('2008', [0]), ('2009', [6]), ('2012', [4, 40, 44]), ('2013', [10]), ('2014', [1, 7, 41]), ('2016', [42]), ('2019', [23, 33, 35, 49]), ('2020', [13, 38, 39, 43]), ('2021', [14, 36, 37, 45, 46, 48, 50]), ('2022', [12, 15, 16, 17, 18, 19, 20, 21, 22, 28, 29, 30, 32, 34]), ('21', [20]), ('22', [21]), ('23', [10, 22]), ('236', [47])

In [ ]:
def getList(term) :
  # prepare additional dictionaries
  ki = dict()
  ik = dict()
  for i, k in enumerate(invertedIndex):
    ki[k] = i   # dictionary index_of_key
    ik[i] = k   # dictionary key_of_index
 
  # initializing key and offset
  current_key = term
  offset = 1  # (1 for next key, but can be any existing distance)
 
  # Get previous key in Dictionary and read length
  pos_before = 0
  if term in ki :
    index_of_current_key = ki[term]
    index_of_previous_key = index_of_current_key - offset
    term_before = ik[index_of_previous_key] if index_of_previous_key in ik else None

    pos = invertedIndex[term]
    if term_before is not None :
      pos_before = invertedIndex[term_before]
      read_len = pos - pos_before
    else :
      pos_before = pos
      read_len = 1
  else :
    read_len = 0

  #Get list from posting lists file
  with open("posting_list.bin", "rb") as binary_file :
    binary_file.seek(pos_before)
    encoded = binary_file.read(read_len)
    decoded = decode(encoded)
  
  return decoded

In [ ]:
def intersection(lst1, lst2):
  lst3 = [value for value in lst1 if value in lst2]
  return lst3
  
def queryProcess(query) :
  query = query.lower()
  query = regexp.tokenize(query)
  query = [word for word in query if word not in stopwords]
  query = [ps.stem(word) for word in query if word not in stopwords]
  intersect_list = getList(query[0])
  for word in query :
    current_list = getList(word)
    intersect_list = intersection(intersect_list, current_list)

  return intersect_list

In [ ]:
query = input("Insert Query : ")
result = queryProcess(query)
print(result)

Insert Query : computer science
[36, 38, 39, 40, 42, 43, 44, 50]
